# Введение в MapReduce модель на Python


In [55]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator
import random

In [56]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [57]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [58]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [59]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [60]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [61]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [62]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [63]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [64]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [65]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [66]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [67]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [68]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [69]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.6255698617172936),
 (1, 1.6255698617172936),
 (2, 1.6255698617172936),
 (3, 1.6255698617172936),
 (4, 1.6255698617172936)]

## Inverted index

In [70]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [71]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [72]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [73]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('it', 18), ('what', 10)]),
 (1, [('is', 18)])]

## TeraSort

In [74]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.08806593405215135),
   (None, 0.20330395628099673),
   (None, 0.23644177535613342),
   (None, 0.26427136800906714),
   (None, 0.2707478355147447),
   (None, 0.2960450822088805),
   (None, 0.29702241927385575),
   (None, 0.3304485094326084),
   (None, 0.3468833581291805),
   (None, 0.42557308150939666),
   (None, 0.44188286199009086),
   (None, 0.4477301363122663),
   (None, 0.4527765340072626),
   (None, 0.4727523467695264),
   (None, 0.4842655140430563)]),
 (1,
  [(None, 0.5063936512097185),
   (None, 0.5162690713568517),
   (None, 0.5163795888986793),
   (None, 0.5716777801411518),
   (None, 0.6800091275557612),
   (None, 0.6950411449585012),
   (None, 0.7002295226684974),
   (None, 0.7096822273965279),
   (None, 0.7104258426935389),
   (None, 0.7496302861146492),
   (None, 0.8033726628654874),
   (None, 0.8399269880287269),
   (None, 0.988696740930004),
   (None, 0.9909896908157181),
   (None, 0.9970259333950889)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [75]:
import random

def MAP(num_list):
    return max(num_list)


def REDUCE(num_list):
    return max(num_list)


def RECORDREADER(count):
    return [random.randint(0, count * 10) for i in range(count)]


record = RECORDREADER(100)
print(record)

part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitional)

print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

[256, 103, 254, 702, 431, 951, 519, 915, 723, 852, 800, 566, 621, 626, 575, 535, 537, 914, 252, 980, 953, 484, 409, 771, 800, 280, 856, 450, 572, 279, 410, 730, 388, 802, 159, 551, 874, 576, 824, 13, 130, 553, 522, 820, 480, 977, 385, 941, 78, 809, 660, 712, 582, 338, 135, 353, 670, 900, 984, 745, 111, 44, 417, 112, 427, 322, 261, 52, 659, 264, 400, 936, 321, 776, 143, 97, 996, 537, 865, 110, 149, 667, 141, 680, 439, 659, 615, 490, 864, 312, 766, 93, 725, 987, 918, 775, 424, 339, 952, 578]
[[256, 103, 254, 702, 431], [951, 519, 915, 723, 852], [800, 566, 621, 626, 575], [535, 537, 914, 252, 980], [953, 484, 409, 771, 800], [280, 856, 450, 572, 279], [410, 730, 388, 802, 159], [551, 874, 576, 824, 13], [130, 553, 522, 820, 480], [977, 385, 941, 78, 809], [660, 712, 582, 338, 135], [353, 670, 900, 984, 745], [111, 44, 417, 112, 427], [322, 261, 52, 659, 264], [400, 936, 321, 776, 143], [97, 996, 537, 865, 110], [149, 667, 141, 680, 439], [659, 615, 490, 864, 312], [766, 93, 725, 987, 918

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [76]:
# ключ группировки одинаковый для всех
def MAP(num):
    return (1, num)


def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]


map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)

[(1, 26), (1, 26), (1, 70), (1, 11), (1, 92), (1, 67), (1, 53), (1, 43), (1, 100), (1, 14), (1, 46), (1, 81), (1, 95), (1, 47), (1, 59), (1, 93), (1, 28), (1, 80), (1, 33), (1, 89), (1, 63), (1, 44), (1, 43), (1, 5), (1, 63), (1, 5), (1, 90), (1, 67), (1, 34), (1, 24), (1, 40), (1, 10), (1, 30), (1, 38), (1, 80), (1, 96), (1, 10), (1, 45), (1, 77), (1, 76), (1, 84), (1, 73), (1, 64), (1, 2), (1, 87), (1, 58), (1, 67), (1, 82), (1, 17), (1, 40), (1, 30), (1, 40), (1, 87), (1, 36), (1, 41), (1, 99), (1, 67), (1, 42), (1, 72), (1, 58), (1, 39), (1, 70), (1, 94), (1, 7), (1, 51), (1, 4), (1, 12), (1, 69), (1, 18), (1, 43), (1, 44), (1, 45), (1, 38), (1, 38), (1, 93), (1, 76), (1, 5), (1, 26), (1, 21), (1, 79), (1, 95), (1, 29), (1, 37), (1, 53), (1, 94), (1, 33), (1, 76), (1, 80), (1, 23), (1, 32), (1, 9), (1, 69), (1, 9), (1, 60), (1, 75), (1, 26), (1, 83), (1, 2), (1, 62), (1, 58)]
[(1, [26, 26, 70, 11, 92, 67, 53, 43, 100, 14, 46, 81, 95, 47, 59, 93, 28, 80, 33, 89, 63, 44, 43, 5, 63, 5

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [77]:
def group_by_key(iterable):
    sorted_data = sorted(iterable, key=lambda x: x[0])
    grouped_data = {}
    for key, value in sorted_data:
        if key not in grouped_data:
            grouped_data[key] = [value]
        else:
            grouped_data[key].append(value)
    result = []
    for key, value in grouped_data.items():
        result.append((key, value))
    return result

def MAP(num):
    return (1, num)


def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)

[(1, 97), (1, 99), (1, 62), (1, 90), (1, 0), (1, 49), (1, 25), (1, 98), (1, 42), (1, 39), (1, 8), (1, 14), (1, 99), (1, 39), (1, 20), (1, 80), (1, 50), (1, 3), (1, 62), (1, 76), (1, 65), (1, 49), (1, 45), (1, 71), (1, 16), (1, 77), (1, 63), (1, 64), (1, 80), (1, 27), (1, 82), (1, 34), (1, 32), (1, 41), (1, 22), (1, 83), (1, 88), (1, 42), (1, 69), (1, 21), (1, 22), (1, 15), (1, 88), (1, 47), (1, 17), (1, 54), (1, 32), (1, 40), (1, 100), (1, 9), (1, 40), (1, 52), (1, 20), (1, 13), (1, 87), (1, 23), (1, 68), (1, 80), (1, 71), (1, 90), (1, 81), (1, 85), (1, 75), (1, 18), (1, 57), (1, 29), (1, 21), (1, 53), (1, 68), (1, 83), (1, 39), (1, 95), (1, 14), (1, 95), (1, 72), (1, 91), (1, 70), (1, 25), (1, 58), (1, 12), (1, 27), (1, 35), (1, 71), (1, 90), (1, 56), (1, 59), (1, 92), (1, 4), (1, 82), (1, 97), (1, 56), (1, 13), (1, 46), (1, 42), (1, 35), (1, 4), (1, 53), (1, 34), (1, 7), (1, 31)]
[(1, [97, 99, 62, 90, 0, 49, 25, 98, 42, 39, 8, 14, 99, 39, 20, 80, 50, 3, 62, 76, 65, 49, 45, 71, 16, 77

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [78]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs


# ===================================================================================================================================================================================

from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, word)

def REDUCE(key:str, value:Iterator[str]):
  yield key

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, ['', 'a', 'banana', 'it', 'what']), (1, ['is'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [79]:
from collections import defaultdict

def MAP(el_list):
    mapped_result = defaultdict(list)
    for t in el_list:
        if C(t):  # Предикат C
            mapped_result[t].append(t)
    return mapped_result.items()


def REDUCE(mapped_items):
    reduced_result = []
    print(mapped_items)
    for values in mapped_items:
        for value in values:
            reduced_result.append(value[0])
    return reduced_result

def C(t):
    return t[0] % 2 == 0  # Выбираем только кортежи с четным первым элементом


def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

record = RECORDREADER(100)
print(record)

part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitional)

print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

[(39, 88, 80), (98, 93, 79), (80, 24, 13), (89, 19, 34), (76, 96, 20), (67, 8, 19), (59, 27, 43), (70, 92, 64), (34, 15, 77), (51, 50, 46), (66, 7, 84), (49, 84, 65), (87, 78, 37), (26, 93, 62), (65, 26, 6), (1, 72, 57), (7, 7, 91), (8, 54, 55), (77, 63, 25), (60, 48, 19), (65, 26, 75), (75, 72, 85), (58, 64, 94), (9, 54, 95), (30, 18, 30), (1, 50, 70), (49, 33, 25), (80, 68, 11), (57, 46, 99), (22, 53, 100), (14, 15, 46), (34, 48, 15), (45, 1, 15), (35, 56, 38), (29, 72, 55), (48, 30, 62), (30, 30, 76), (87, 64, 46), (59, 68, 28), (27, 98, 68), (27, 67, 1), (28, 14, 65), (41, 79, 48), (22, 13, 26), (24, 85, 33), (100, 88, 82), (14, 56, 70), (0, 65, 84), (49, 7, 27), (81, 14, 70), (64, 62, 83), (35, 94, 0), (15, 48, 68), (46, 51, 61), (44, 0, 84), (70, 53, 2), (6, 0, 65), (86, 51, 43), (86, 17, 46), (0, 94, 34), (86, 5, 86), (26, 29, 66), (45, 36, 78), (13, 40, 73), (76, 23, 21), (22, 4, 71), (62, 55, 5), (84, 25, 23), (48, 72, 92), (31, 29, 66), (13, 9, 23), (95, 96, 44), (77, 99, 37)

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [80]:
S = set([1 ,12, 56])

def MAP(t):
  res_list = []
  for el in t:
    if el in S:
      res_list.append(el)
    res = tuple(res_list)
    return (res, res)


def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print(reduce_output)

[(85, 34, 81), (55, 44, 15), (41, 77, 32), (22, 6, 36), (8, 33, 6), (68, 56, 60), (18, 12, 0), (40, 33, 83), (69, 78, 13), (76, 45, 38), (18, 46, 21), (34, 28, 67), (81, 78, 95), (3, 95, 29), (50, 97, 22), (4, 33, 15), (70, 73, 70), (51, 57, 48), (6, 30, 94), (43, 12, 31), (57, 67, 37), (54, 10, 0), (72, 23, 1), (52, 11, 64), (29, 11, 44), (51, 1, 19), (12, 42, 83), (88, 7, 36), (82, 8, 98), (17, 50, 4), (99, 94, 28), (93, 53, 24), (14, 62, 60), (81, 81, 78), (33, 20, 84), (42, 17, 97), (48, 49, 18), (32, 77, 77), (21, 24, 64), (87, 98, 82), (82, 82, 22), (11, 23, 38), (6, 78, 84), (34, 75, 45), (13, 32, 28), (11, 74, 79), (50, 47, 13), (1, 46, 6), (1, 57, 40), (10, 74, 99), (68, 34, 99), (71, 95, 17), (84, 89, 8), (50, 93, 32), (50, 52, 75), (78, 3, 71), (81, 9, 3), (32, 99, 33), (74, 70, 12), (23, 81, 77), (80, 58, 86), (95, 29, 30), (29, 19, 39), (73, 36, 27), (69, 35, 79), (89, 39, 72), (81, 6, 71), (97, 13, 44), (92, 7, 63), (92, 30, 35), (63, 50, 65), (16, 20, 18), (20, 89, 0), (

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [81]:
def MAP(t):
  return (t, t)


def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print(reduce_output)

[(54, 84, 5), (62, 29, 67), (76, 2, 7), (22, 7, 40), (51, 92, 77), (11, 70, 86), (82, 6, 60), (90, 0, 73), (60, 57, 91), (67, 70, 19), (69, 100, 16), (71, 71, 95), (25, 32, 13), (31, 14, 49), (1, 19, 82), (43, 35, 22), (74, 35, 45), (46, 24, 84), (35, 67, 64), (2, 64, 17), (92, 59, 33), (30, 87, 85), (66, 1, 20), (8, 36, 78), (10, 59, 35), (93, 84, 61), (22, 6, 7), (0, 3, 59), (42, 48, 35), (75, 35, 76), (58, 50, 85), (77, 63, 5), (24, 15, 28), (17, 32, 62), (30, 9, 96), (54, 20, 25), (91, 76, 74), (25, 50, 48), (60, 35, 30), (25, 58, 71), (59, 80, 20), (44, 93, 18), (11, 55, 1), (54, 60, 23), (8, 63, 3), (8, 12, 51), (23, 72, 39), (9, 69, 20), (24, 26, 70), (30, 95, 81), (28, 37, 51), (51, 4, 39), (60, 76, 9), (66, 94, 61), (8, 1, 36), (29, 40, 56), (5, 31, 61), (43, 58, 90), (4, 20, 56), (58, 50, 42), (94, 2, 31), (21, 44, 3), (61, 84, 68), (68, 72, 48), (83, 71, 68), (94, 46, 10), (60, 40, 8), (81, 10, 31), (14, 85, 62), (36, 23, 23), (2, 32, 83), (0, 27, 47), (3, 86, 80), (13, 34, 

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [82]:
def MAP(t):
  return (t, t)


def REDUCE(key, values: Iterator[NamedTuple]):
  if len(values) == 2:
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print(reduce_output)

[(3, 3), (3, 3), (0, 2), (1, 2), (2, 0), (0, 0), (3, 1), (2, 3), (2, 2), (3, 3), (0, 1), (1, 3), (1, 1), (0, 0), (0, 2), (2, 1), (0, 1), (1, 3), (3, 3), (1, 2), (2, 1), (3, 2), (2, 1), (1, 2), (0, 1), (0, 1), (2, 2), (0, 0), (1, 1), (1, 2), (0, 0), (0, 0), (3, 2), (3, 3), (1, 3), (1, 0), (3, 2), (1, 0), (2, 1), (1, 2), (1, 0), (1, 0), (2, 0), (3, 3), (0, 2), (1, 1), (3, 1), (3, 2), (3, 2), (1, 2), (1, 2), (1, 1), (3, 1), (2, 1), (2, 3), (2, 0), (2, 0), (0, 2), (1, 0), (0, 3), (0, 1), (2, 1), (3, 0), (0, 0), (0, 0), (3, 2), (1, 0), (0, 3), (0, 1), (3, 0), (2, 2), (0, 0), (2, 1), (2, 0), (3, 3), (0, 0), (0, 3), (0, 2), (3, 2), (3, 0), (0, 3), (3, 0), (1, 3), (1, 1), (3, 0), (2, 1), (3, 0), (3, 1), (3, 0), (3, 3), (1, 3), (3, 0), (3, 0), (2, 2), (1, 1), (0, 0), (2, 3), (3, 1), (2, 1), (3, 0)]
[((2, 3), (2, 3)), ((0, 3), (0, 3)), ((3, 0), (3, 0)), ((0, 3), (0, 3)), ((2, 1), (2, 1)), ((0, 2), (0, 2)), ((1, 2), (1, 2)), ((0, 1), (0, 1)), ((1, 2), (1, 2)), ((2, 3), (2, 3)), ((1, 1), (1, 1)), 

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [83]:
rels = [1, 2]
class Tuple: # класс кортежа, который хранит данные и имя представления из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple(count):
  data = tuple([(random.randint(0, 3), random.randint(0, 3)) for i in range(count)])
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)


def RECORDREADER(count):
    return [get_random_tuple(3) for i in range(count)]

def MAP(t: Tuple):
  return (t.data, t.rel_id)

def REDUCE(key, values: Iterator[NamedTuple]):
  if values == [rels[0]]: # если соответствующее значение является списком  [R] , создайте пару  (t,t)
    return (key, key)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print(reduce_output)

[<__main__.Tuple object at 0x7b4b7c93d060>, <__main__.Tuple object at 0x7b4b7c93da20>, <__main__.Tuple object at 0x7b4b7c93d240>, <__main__.Tuple object at 0x7b4b7c93d3c0>, <__main__.Tuple object at 0x7b4b7c93da80>, <__main__.Tuple object at 0x7b4b7c93f880>, <__main__.Tuple object at 0x7b4b7c93f070>, <__main__.Tuple object at 0x7b4b7c93ca30>, <__main__.Tuple object at 0x7b4b7c93e170>, <__main__.Tuple object at 0x7b4b7c93e0b0>, <__main__.Tuple object at 0x7b4b7c93fd00>, <__main__.Tuple object at 0x7b4b7c93d9c0>, <__main__.Tuple object at 0x7b4b7c93d540>, <__main__.Tuple object at 0x7b4b7c93cbe0>, <__main__.Tuple object at 0x7b4b7c93cc40>, <__main__.Tuple object at 0x7b4b7c93ebc0>, <__main__.Tuple object at 0x7b4b7c93ec50>, <__main__.Tuple object at 0x7b4b7c93ccd0>, <__main__.Tuple object at 0x7b4b7c93dcf0>, <__main__.Tuple object at 0x7b4b7c93c700>, <__main__.Tuple object at 0x7b4b7c93c6a0>, <__main__.Tuple object at 0x7b4b7c93c0a0>, <__main__.Tuple object at 0x7b4b7c93c670>, <__main__.

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [84]:
rels = [1, 2]
class Tuple: # класс кортежа, который хранит данные и имя представления из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple():
  data = (random.randint(0, 3), random.randint(0, 3))
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)


def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  if t.rel_id == rels[0]:
    return (t.data[1], (t.rel_id, t.data[0]))
  else:
    return (t.data[0], (t.rel_id, t.data[1]))

def REDUCE(key, values: Iterator[NamedTuple]):
  res = []
  for v in values:
    res.append((v[0], key, v[1]))
  return res

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print(reduce_output)

[<__main__.Tuple object at 0x7b4b7ca1efe0>, <__main__.Tuple object at 0x7b4b7ca1fdf0>, <__main__.Tuple object at 0x7b4b7ca1f6d0>, <__main__.Tuple object at 0x7b4b7ca1d360>, <__main__.Tuple object at 0x7b4b7ca1d5d0>, <__main__.Tuple object at 0x7b4b7ca1d9c0>, <__main__.Tuple object at 0x7b4b7ca1c940>, <__main__.Tuple object at 0x7b4b7ca1c2b0>, <__main__.Tuple object at 0x7b4b7ca1c190>, <__main__.Tuple object at 0x7b4b7ca1f790>, <__main__.Tuple object at 0x7b4b7ca1e800>, <__main__.Tuple object at 0x7b4b7ca1f760>, <__main__.Tuple object at 0x7b4b7ca49db0>, <__main__.Tuple object at 0x7b4b7ca4ad10>, <__main__.Tuple object at 0x7b4b7ca48160>, <__main__.Tuple object at 0x7b4b7ca4ae90>, <__main__.Tuple object at 0x7b4b7ca4a080>, <__main__.Tuple object at 0x7b4b7ca4ae60>, <__main__.Tuple object at 0x7b4b7ca49c90>, <__main__.Tuple object at 0x7b4b7ca4ae30>, <__main__.Tuple object at 0x7b4b7ca48df0>, <__main__.Tuple object at 0x7b4b7ca49060>, <__main__.Tuple object at 0x7b4b7ca48400>, <__main__.

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [85]:
def get_random_tuple():
  return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  return (t[0], t[1])

def tetta(values):
  return sum(values)


def REDUCE(key, values: Iterator[NamedTuple]):
  x = tetta(values)
  return (key, x)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print(reduce_output)

[(3, 1, 1), (0, 1, 1), (1, 0, 3), (1, 0, 3), (1, 1, 2), (2, 0, 2), (2, 2, 1), (1, 3, 2), (3, 2, 3), (2, 1, 3), (3, 3, 3), (0, 0, 1), (2, 0, 2), (1, 1, 0), (3, 2, 2), (3, 2, 1), (2, 0, 0), (1, 2, 1), (2, 1, 1), (1, 0, 3), (3, 3, 2), (2, 3, 1), (0, 2, 3), (2, 3, 1), (0, 3, 3), (2, 2, 2), (0, 1, 3), (2, 3, 3), (0, 0, 1), (3, 0, 0), (2, 0, 3), (1, 1, 0), (2, 1, 3), (3, 0, 1), (2, 3, 0), (2, 1, 3), (3, 1, 1), (3, 1, 0), (2, 3, 0), (1, 1, 0), (1, 3, 0), (3, 3, 2), (0, 0, 1), (1, 0, 2), (2, 3, 0), (1, 3, 3), (0, 1, 3), (0, 0, 0), (3, 1, 3), (0, 0, 1), (2, 2, 3), (2, 0, 3), (2, 2, 1), (3, 1, 2), (2, 0, 2), (2, 3, 0), (0, 0, 0), (0, 3, 3), (1, 2, 0), (1, 1, 0), (3, 3, 3), (1, 0, 0), (0, 2, 0), (2, 0, 1), (2, 2, 1), (0, 3, 2), (2, 1, 1), (0, 2, 0), (0, 1, 0), (1, 2, 2), (0, 2, 3), (0, 3, 2), (3, 3, 0), (0, 2, 0), (0, 2, 1), (0, 1, 0), (3, 0, 2), (2, 1, 1), (3, 1, 2), (1, 0, 3), (2, 1, 3), (0, 0, 0), (1, 2, 0), (3, 0, 0), (3, 3, 3), (0, 0, 0), (1, 0, 0), (2, 0, 0), (3, 0, 2), (1, 0, 0), (3, 3, 2)

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [86]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [87]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j, k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
  (i, k) = key
  el_value = 0
  for v in values:
    el_value += v
  yield ((i, k), el_value)

Проверьте своё решение

In [88]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [89]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [90]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def RECORDREADER():
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      yield ((0, i, j), small_mat[i,j]) # первая матрица

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((1, j, k), big_mat[j,k]) # вторая матрица

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def MAP_MUL(k1, v1):
  (i, k) = k1
  yield ((i, k), v1)


def REDUCE_MUL(key, values):
  res_el_value = 0
  for v in values:
    res_el_value += v
  yield (key, res_el_value)


def GET_JOINED():
  for j in joined:
    yield j

joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)
np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [91]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# ===================================================================================================================================================================================

import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) # первая матрица
  yield first_mat

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) # вторая матрица
  yield second_mat


def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]


def MAP_MUL(k1, v1):
  yield (k1, v1)


def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 4
reducers = 2
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution)) # should return true


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.004882209977262622), ((0, 1), 0.0038683451450971613), ((0, 2), 0.003390957747262682), ((0, 3), 0.003214427472294149), ((0, 4), 0.0036018560350790443), ((0, 5), 0.0009688484558747747), ((0, 6), 0.004937945352445722), ((0, 7), 0.00024106855149476318), ((0, 8), 0.0036262309305828667), ((0, 9), 0.0028645230944768893), ((0, 10), 0.0029622096829746633), ((0, 11), 0.000612627135925204), ((0, 12), 0.004594738074563853), ((0, 13), 0.0033357819100331572), ((0, 14), 0.0005310222298188597), ((0, 15), 0.0049873557765213045), ((0, 16), 0.0026101958816150484), ((0, 17), 0.0052606402573071335), ((0, 18), 0.003688775428009355), ((0, 19), 0.0035654339981394064), ((0, 20), 0.0011257624299577998), ((0, 21), 0.00515052073444553), ((0, 22), 0.0007713453302701629), ((0, 23), 0.0032375162015068435), ((0, 24), 0.003396008622807865), ((0, 25), 0.0018057050375171655), ((0, 26), 0.004540959918195643), ((0, 27), 0.0026251648417825128), ((0, 28), 0.0049

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [92]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# ===================================================================================================================================================================================

import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) # первая матрица

  global maps
  split_size =  int(np.ceil(len(first_mat)/maps))
  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) # вторая матрица

  split_size =  int(np.ceil(len(second_mat)/maps))
  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]


def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]


def MAP_MUL(k1, v1):
  yield (k1, v1)


def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution)) # should return true


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.2597241050229439), ((0, 1), 0.401575155675791), ((0, 2), 0.48696056075347904), ((0, 3), 0.15550329099934523), ((0, 4), 0.6474419040648105), ((0, 5), 0.21157506698264378), ((0, 6), 0.5493977409722797), ((0, 7), 0.12556834078574267), ((0, 8), 0.48166764407265433), ((0, 9), 0.14121495297756564), ((0, 10), 0.21719559880892286), ((0, 11), 0.44477801190453703), ((0, 12), 0.46100539969532117), ((0, 13), 0.6762059090446996), ((0, 14), 0.33092172487529037), ((0, 15), 0.15605497432927276), ((0, 16), 0.0017540311672767492), ((0, 17), 0.07559319004211063), ((0, 18), 0.5329479878291786), ((0, 19), 0.33301178653165975), ((0, 20), 0.38581091716718885), ((0, 21), 0.5607321278530568), ((0, 22), 0.647414618432631), ((0, 23), 0.13884507959910106), ((0, 24), 0.6556002174363336), ((0, 25), 0.6164376851333723), ((0, 26), 0.5012853954486918), ((0, 27), 0.5336417651420756), ((0, 28), 0.6648073794634358), ((0, 29), 0.10175440197720324), ((0, 30), 0

True